# 오류 처리 (Error Handling)
## 스위프트는 어떻게 오류를 처리할까?
## 오류 처리 구문

## 오류 던지기

### 자신이 작성한 오류 타입 객체는 함수나 메소드를 실행하는 과정에서 필요에 따라 외부로 던져 실행 흐름을 옮겨버릴 수 있다
- #### 이때 함수나 메소드는 오류 객체를 외부로 던질 수 있다는 것을 컴파일러에 알려주기 위해 정의 구문을 작성할 때 throws 키워드를 추가한다

### throws 키워드는 반환 타입을 표시하는 화살표 ('->')보다 앞에 작성
- #### 이는 오류를 던지면 값이 반환되지 않는다는 의미이기도 하다

### 함수나 메소드, 또는 익명 함수인 클로저까지 모두 throws 키워드를 사용할 수 있지만,
- #### 명시적으로 throws 키워드를 추가하지 않으면 오류를 던질 수 없다

```
func canThrowsErrors() throws -> String
func cannotThrowErrors() -> String
```

### 위 두 함수는 모두 문자열을 반환하지만, 
- #### canThrowErrors() 함수는 실행 과정에서 오류가 발생하면 그 오류를 객체로 만들어 던질 수 있다
- #### cannotThrowErrors() 메소드는 오류가 발생하더라도 오류 객체를 던질 수 없다

### 클로저를 이용하여 구문을 작성할 때도 이와 비슷하게 반환 화살표 앞에 작성한다

```
{() throws -> String in
    ...
}
```


### 이렇게 throws 키워드가 추가된 함수나 메소드, 또는 클로저는 실행 블록 어느 지점에서건 우리가 의도하는 오류를 던질 수 있다

### 오류를 실제로 던질 때는 throw 키워드를 사용하는데,
- #### 이는 함수나 메소드 등에서 오류를 던진다는 것을 선언할 때 사용했던 throws 키워드와 비슷하지만 단수라는 사실에 주의해야 한다

### 실제로 날짜를 분석하는 함수를 작성하고, 실행 과정에서 발생할 수 있는 오류 상황에서 오류 객체를 던져보자
- #### 앞서 작성한 DaeParseError 오류 객체를 사용

```
import Foundation

struct Date {
    var year: Int
    var month: Int
    var date: Int
}


func parseDate(param: NSString) throws -> Date {
    // 입력된 문자열의 길이가 10이 아닐 경우 분석이 불가능하므로 오류
    guard param.lenght == 10 else {
        if param.length > 10 {
            throw DateParseError.overSizeString
        } else {
            throw DateParseError.underSizeString
        }
    }
    
    
    // 반환할 객체 타입 선언
    var dateResult = Date(year: 0, month: 0, date: 0)
    
    
    // 연도 정보 분석
    if let year = Int(param.substring(with: NSRange(location: 0, length: 4))) {
        dateResult.year = year
    } else {
        // 연도 분석 오류
        throw DateParseError.incorrectFormat(part: "year")
    }
    
    // 월 정보 분석
    if let month = Int(param.substring(with: NSRange(location: 5, length: 2))) {
        // 월에 대한 값은 1 ~ 12까지만 가능하므로 그 이외의 범위는 잘못된 값으로 처리한다.
        guard month > 0 && month < 13 else {
            throw DateParseError.incorrectData(part: "month")
        }
        dateResult.month = month
    } else {
        // 월 분석 오류
        throw DateParseError.incorrectFormat(part: "month")
    }
    
    
    // 일 정보 분석
    if let date = Int(param.substring(with: NSRange(location: 8, length: 2))) {
        // 일에 대한 값은 1 ~ 31까지만 가능하므로 그 이외의 범위는 잘못된 값으로 처리한다.
        guard date > 0 && date < 32 else {
            throw DateParseError.incorrectData(part: "date")
        }
        dateResult.date = date
    } else {
        // 일 분석 오류
        throw DateParseError.incorrectFormat(part: "date")
    }
    
    
    return dateResult
    
}
```

### 위 예제에서 가장 먼저 살펴볼 부분은 파운데이션 라이브러리를 호출하는 import Foundation 구문이다
- #### 문자열 분석의 편의를 위해 스위프트에서 기본으로 제공하는 String 타입이 아닌 NSString 타입의 문자열을 사용할 예정
    - #### 이 타입은 파운데이션 프레임워크에서 제공하는 것으로서 String 타입보다 사용할 수 있는 메소드가 다양하다
- #### 물론 String 타입과 NSString 타입은 서로 호환된다

### 여기에 더해서 인자값으로 사용하는 문자열 일부분을 잘라내기 위해 NSRange 객체도 사용해야 하는데,
- #### 이 역시 파운데이션 프레임워크에 정의되어 있는 객체이다
- #### 앱을 만들거나 코드를 작성하면 객체의 이름 앞에 NS 접두어가 붙은 객체를 봤을 땐 '파운데이션 프레임워크를 호출'하고 생각

### 이어서 정의된 것은 날짜 정보를 담아서 반환할 Date 구조체이다
- #### 날짜 정보는 각각 연도와 월, 일로 이루어지므로 이들을 모두 모아서 반환할 수 있도록 구조체를 선언하였다
- #### 이어지는 함수에서는 날짜 분석을 끝낸 결과값을 Date 구조체에 담아 반환한다
- #### 이 구조체는 각각의 결과값들을 담을 수 있도록 year, month, date 세 개의 프로퍼티를 가지고 있다

### 세 번째로 parseDate(param:) 함수가 정의되어 있는데,
- #### 이 함수가 실제로 입력된 문자열을 분석하여 Date 객체에 적절한 값을 담은 뒤 반환하는 함수이다
- #### 필요한 대부분 구문이 이 함수에서 작성되며, 분석 과정에서 문제가 있을 경우 오류를 던지기 위해 함수를 정의할 때 throws 키워드를 추가로 작성해둔 것을 볼 수 있다

### parseDate(param:) 함수가 호출되면 가장 먼저 수행하는 내용은 입력된 문자열의 길이 점검이다
- #### guard 구문을 이용하여 점검하는데, guard 구문에서는 일단 인자값 문자열의 길이가 정확하게 10일 때만 구문을 계속 진행할 수 있도록 하고 그렇지 않으면 오류를 던진다
- #### 정의한 오류 DateParseError에서는 입력값이 필요한 크기보다 클 때와 작을 때를 나누어 오류를 던질 수 있도록 세분하고 있으므로 여기에서도 입력된 문자열의 길이를 점검하여 다음과 같이 오류를 반환할 수 있도록 if 구문을 사용하고 있다
    - #### 문자열 길이가 10을 초과하면 DateParseError.overSizeString 오류
    - #### 문자열 길이가 10 미만이면 DateParseError.underSizeSrting 오류

### 입력된 문자열의 길이가 정확히 10이라면
- #### guard 구문을 통과하여 다음 단계로 진행
- #### 함수의 실행 결과로 반환될 Date 객체의 인스턴스를 생성한 다음, 본격적으로 문자열을 분석하는 과정으로 넘어가게 된다

### 가장 먼저 분석하는 것은 연도 정보이다
- #### 문자열을 앞에서부터 4개까지만 읽어온 후, 이를 정수로 반환하여 연도 정보로 만들어야 한다
- #### 문자열을 원하는 위치에서 잘라내려면 substring(with:) 메소드를 사용한다
    - #### 이 메소드는 범위 정보 처리하는 NSRange 객체를 인자값으로 받는다
- #### NSRange 객체의 여러 초기화 구문 형식 중에서 사용해야 할 것을 NSRange(location:length:)이다
    - #### 이는 시작 위치로부터 원하는 길이만큼의 범위 객체를 만들어주는 구문이다

### 이를 이용하여 0번째 위치부터 4만큼의 길이 범위를 가지는 NSRange 객체를 만든 다음 이를 substring(with:) 메소드의 인자값으로 넣어주면 입력된 문자열로부터 원하는 연도 네 자리 문자열을 추출할 수 있다
- #### 이 값을 정수로 변환하여 Date 객체의 year 프로퍼티에 할당해주면 연도에 대한 분석 작업이 완료된다

### 이 과정에서 만약 0~4까지의 문자열이 숫자가 아니거나 숫자로 변환할 수 없는 형식이라면 정수가 반환되는 대신 옵셔널 타입인 nil이 반환된다
- #### 이를 점검하여 DateParseError.incorrectFormat 오류를 반환해주면 함수를 호출한 외부에서는 무슨 오류가 발생했는지 쉽게 알 수 있을 것이다
- #### 특히 잘못된 포맷에 의한 오류를 좀 더 자세히 설명하고자 연관 데이터를 추가할 수 있도록 오류 타입이 정의되어 있으므로 이를 활용하면 더욱 자세한 오류 정보를 던질 수 있다

### 이어서 월과 날자에 대한 분석이 차례로 진행된다
- #### 분석 과정 자체는 연도를 분석한 방식과 같지만, 월은 입력받을 수 있는 값이 1부터 12까지로 제한되므로 적절한 값인지를 검사하는 과정이 guard 구문으로 추가되었다
- #### 월에 할당된 값이 0보다 크면서 13보다 작은, 즉 1에서 12까지일 때만 프로세스를 계속 진행하고, 그렇지 않으면 DateParseError.incorrectData 오류를 던진다

### 날짜도 이와 비슷하다
- #### 입력받을 수 있는 값이 1부터 31까지로 제한되므로 이에 대한 검사 과정이 guard 구문으로 추가되었다
- #### 사실 날짜를 점검할 때는 2월의 경우 날짜가 28일을 넘어가는 값으로 입력되면 마찬가지로 오류를 던지도록 처리해야 맞겠지만, 여기에서는 그에 해당하는 점검은 생략하였다

### 모든 과정이 문제없이 진행되면 연-월-일 정보가 저장된 Date 객체가 완성된다
- #### 이 값을 반환하면 함수의 역활은 끝난다

### 이렇게 정의된 함수나 메소드를 호출할 때는 다음과 같이 호출할 이름 앞에 try 키워드를 붙여야 한다

```
try parseData(param: "2020-02-28")
```

### 이 값을 다른 변수나 상수에 할당할 때도 try 키워드는 항상 함수의 이름 바로 앞에 있어야 한다

```
try date = try parseDate(param: "2020-02-28")
```

### 만약 try 키워드 없이 함수를 호출하면 컴파일러에 의한 오류가 발생한다
- #### 물론 오류를 던지지 않는 일반 함수는 try 키워드가 필요 없다
- #### 하지만 try 키워드는 단순히 함수를 호출할수만 있을 뿐 함수에서 던지는 오류를 잡아내지는 못한다
- #### 함수에서 던지는 오류를 잡아내려면 catch 구문을 사용해야 한다